In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive

# Mount Google Drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Install the "datasets" package from Hugging face
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.8 MB/s eta 0:00:00


In [ ]:
from datasets import list_datasets, load_dataset

indonlu = load_dataset("indonlp/indonlu", 'smsa')
gplay = load_dataset("jakartaresearch/google-play-review").remove_columns('stars')

Generating train split:   0%|          | 0/11000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1260 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset indonlu downloaded and prepared to /root/.cache/huggingface/datasets/indonlp___indonlu/smsa/1.0.0/4f6bd3985d38bfa901609cc151c052caaa4612322688a67e5aec267a78e930a0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7028 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3012 [00:00<?, ? examples/s]

Dataset google-play-review downloaded and prepared to /root/.cache/huggingface/datasets/jakartaresearch___google-play-review/default/1.0.0/df84e67f495cc6639ab0bbf74ff0190498a0b22294fdaca26a5b25e090671c29. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Convert datasets to Pandas DataFrames
train_indonlu = pd.DataFrame(indonlu['train'])
val_indonlu = pd.DataFrame(indonlu['validation'])
test_indonlu = pd.DataFrame(indonlu['test'])

In [ ]:
train_indonlu.groupby('label').size()

label
0    6416
1    1148
2    3436
dtype: int64

In [ ]:
train_gplay = pd.DataFrame(gplay['train'])
val_gplay = pd.DataFrame(gplay['validation'])
# test_gplay = pd.DataFrame(gplay['test'])

* 0 Positive
* 1 Neutral
* 2 Negative

In [ ]:
# Map labels in 'train_gplay' and 'val_gplay' to binary values
train_gplay.label = train_gplay.label.map({"pos":0, 'neg':2}) 
val_gplay.label = val_gplay.label.map({"pos":0, 'neg':2}) 

### Define the oversampling function

In [ ]:
def oversample(df, df2, index):
  target_size = df.groupby('label').size()[0]
  remaining = target_size - df.groupby('label').size()[index]

  # 1 label
  new = pd.DataFrame([[x,y] for x, y in zip(df2['text'], df2['label']) if y==index], columns=['text','label'])
  if remaining > new.shape[0]:
    remaining = new.shape[0]

  print(remaining, new.shape)
  df = pd.concat([df, new.sample(remaining, replace=False)], ignore_index=True)

  return df

In [ ]:
# Perform oversampling on 'train_indonlu' with label index 1 and 2
train = train_indonlu

train = oversample(train, train_twit, 1)
train = oversample(train, train_twit, 2)

train = oversample(train, train_gplay, 1)
train = oversample(train, train_gplay, 2)

4262 (4262, 2)
2318 (2318, 2)
0 (0, 2)
662 (1141, 2)


In [ ]:
# Perform oversampling on 'val_indonlu' with label index 1 and 2
val = val_indonlu
val = oversample(val, val_gplay, 1)
val = oversample(val, val_gplay, 2) 

val = oversample(val, val_twit, 1)
val = oversample(val, val_twit, 2) 

0 (0, 2)
341 (480, 2)
604 (1065, 2)
0 (569, 2)


In [ ]:
test = test_indonlu

In [ ]:
train.groupby('label').size()

label
0    6416
1    5410
2    6416
dtype: int64

In [ ]:
val.groupby('label').size()

label
0    735
1    735
2    735
dtype: int64

In [ ]:
test.groupby('label').size()

label
0    208
1     88
2    204
dtype: int64

### Text Cleaning Preprocessing

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

import json 

api_token = {"username":"dafagaming123","key":"26d41639614b8fc75387801be2754587"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d oswinrh/indonesian-stoplist
!unzip -n indonesian-stoplist.zip

stoplist = pd.read_csv("stopwordbahasa.csv").to_numpy()

mkdir: cannot create directory ‘/root/.kaggle’: File exists
indonesian-stoplist.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  indonesian-stoplist.zip


In [ ]:
!pip install -q Sastrawi

from tensorflow.keras.preprocessing.text import text_to_word_sequence
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Text Preprocessing
def preprocess(text):
  sentence = text.lower()
  words = text_to_word_sequence(sentence)
  words =  [word for word in words if len(word) > 2 and word.isalnum()]
  words = [word for word in words if word not in stoplist]

  words     = ' '.join(words)
  sentence  =  stemmer.stem(sentence)

  return sentence

# Preprocess dataframe
def df_preprocess(df):
  df['text_clean'] = df['text'].apply(lambda text_string: preprocess(text_string))

  return df

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 13.4 MB/s eta 0:00:00


In [ ]:
train = df_preprocess(train)
val = df_preprocess(val)
test = df_preprocess(test)

Since that test dataset doesnt affect into the accuracy of the model, i better to let it be

In [ ]:
train.to_csv('/content/train_balanced.csv', index=False)
val.to_csv('/content/val_balanced.csv', index=False)
test.to_csv('/content/test_balanced.csv', index=False)

!ls /content

indonesian-stoplist.zip  stopwordbahasa.csv  train_balanced.csv
sample_data		 test_balanced.csv   val_balanced.csv


### Merge Dataset

In [ ]:
merged = pd.concat([train, val, test])
merged.shape

(20947, 3)

### Uploading to Google Drive

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

!ls

# Create & upload a text file.
uploaded_train = drive.CreateFile()
uploaded_train.SetContentFile('train_balanced.csv')
uploaded_train.Upload()

uploaded_val = drive.CreateFile()
uploaded_val.SetContentFile('val_balanced.csv')
uploaded_val.Upload()

uploaded_test = drive.CreateFile()
uploaded_test.SetContentFile('test_balanced.csv')
uploaded_test.Upload()

# Print file IDs
print("[\"{}\", \"{}\", \"{}\"]".format(uploaded_train.get('id'), uploaded_val.get('id'), uploaded_test.get('id')))


indonesian-stoplist.zip  stopwordbahasa.csv  train_balanced.csv
sample_data		 test_balanced.csv   val_balanced.csv
["1yWbvg1gVbSYvlFXahJkeBjHtfNAfwfJG", "1mxU7-H1ROaobkQ8cWqWBSewldisriUfk", "1UjTKjVR4WcsPJzVdNYsdzDGl5wpDoPVE"]
